# Youtube Automation bot
### This bot when run will email you the summary of the latest video of a channel on youtube.

In [13]:
!pip install google-api-python-client youtube-transcript-api transformers

In [20]:
import os
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi,NoTranscriptFound
from transformers import pipeline




# YouTube Data API setup
API_KEY = 'api key here'
CHANNEL_ID = 'UCoMzWLaPjDJBbipihD694pQ'  # Replace this with the actual channel ID of StockMoe
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Initialize the summarization pipelines for BART and T5
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
t5_summarizer = pipeline("summarization", model="t5-small")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [21]:
# Email setup
EMAIL_ADDRESS = 'sender@gmail.com'
EMAIL_PASSWORD = 'apppasskey'
RECIPIENT_EMAIL = 'recipient@gmail.com'

SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587  # Use 465 if you want to use SSL instead of TLS

In [25]:
def get_latest_video():
    request = youtube.search().list(
        part='snippet',
        channelId=CHANNEL_ID,
        maxResults=1,
        order='date'
    )
    response = request.execute()
    if response['items']:
        latest_video = response['items'][0]
        video_id = latest_video['id']['videoId']
        video_title = latest_video['snippet']['title']
        video_url = f'https://www.youtube.com/watch?v={video_id}'
        return video_title, video_url, video_id
    return None, None, None

def get_video_transcript(video_id):
    try:
        # Try to get the official transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
    except NoTranscriptFound:
        # If no official transcript, try auto-generated captions
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        except NoTranscriptFound:
            print(f"No transcript or auto-generated captions found for video ID {video_id}.")
            return None

    transcript_text = " ".join([entry['text'] for entry in transcript])
    return transcript_text

def summarize_text_with_bart(text):
    # Truncate the text if it's too long for the model
    max_input_length = 10240  # Adjust this value based on your model
    text = text[:max_input_length]

    # Clean the text to remove potential problematic characters
    text = text.replace('\n', ' ')

    summary = bart_summarizer(text, max_length=30000, min_length=60, do_sample=False)
    return summary[0]['summary_text']

def summarize_text_with_t5(text):
    # Truncate the text if it's too long for the model
    max_input_length = 10240  # Adjust this value based on your model
    text = text[:max_input_length]

    # Clean the text to remove potential problematic characters
    text = text.replace('\n', ' ')
    summary = t5_summarizer(text, max_length=200, min_length=60, do_sample=False)
    return summary[0]['summary_text']

def format_summary(summary, video_title):
    # Basic text processing to identify key parts
    lines = summary.split('. ')

    # Headline and subtitle
    headline = f"Headline: {video_title}"
    subtitle = f"Subtitle: {lines[0] if len(lines) > 0 else 'Summary of the market and stock signals.'}"

    # Organize into sections
    stock_signals = []
    market_commentary = []

    for line in lines[1:]:
        if "buy" in line.lower() or "sell" in line.lower() or "hold" in line.lower():
            stock_signals.append(line.strip())
        else:
            market_commentary.append(line.strip())

    # Format the output
    formatted_summary = f"{headline}\n{subtitle}\n\nStocks and Signals:\n"
    formatted_summary += '\n'.join(f"- {signal}" for signal in stock_signals)
    formatted_summary += "\n\nMarket Commentary:\n"
    formatted_summary += ' '.join(market_commentary)

    return formatted_summary

def send_email(subject, body):
    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = RECIPIENT_EMAIL
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    server.sendmail(EMAIL_ADDRESS, RECIPIENT_EMAIL, msg.as_string())
    server.quit()

def check_new_video(summarizer='bart'):

    last_video_file = 'last_video.txt'
    print(f"Video checked")

    if os.path.exists(last_video_file):
        with open(last_video_file, 'r') as file:
            last_video_url = file.read().strip()
    else:
        last_video_url = ''

    video_title, video_url, video_id = get_latest_video()

    if video_url and video_url != last_video_url:
        transcript = get_video_transcript(video_id)
        if transcript:
            if summarizer == 'bart':
                summary = summarize_text_with_bart(transcript)
            elif summarizer == 't5':
                summary = summarize_text_with_t5(transcript)
            formatted_summary = format_summary(summary, video_title)
        else:
            formatted_summary = "No transcript or captions available for summarization."

        email_body = f"Check out the new video: {video_url}\n\n{formatted_summary}"
        send_email(f'New Video Uploaded: {video_title}', email_body)

        with open(last_video_file, 'w') as file:
            file.write(video_url)



# Run the check only once
# Example Usage
# To use BART for summarization
check_new_video(summarizer='bart')



Video checked


Your max_length is set to 30000, but your input_length is only 213. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=106)


In [27]:
# To use T5 for summarization
check_new_video(summarizer='t5')

Video checked


Token indices sequence length is longer than the specified maximum sequence length for this model (2938 > 512). Running this sequence through the model will result in indexing errors
